In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--headless')  #ẩn
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-extensions");
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
url='https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/infectious-and-parasitic-diseases'

list_url=['https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/infectious-and-parasitic-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/respiratory-infections',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/maternal-conditions',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/perinatal-conditions',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/nutritional-deficiencies',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/malignant-neoplasms',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/diabetes-mellitus-and-endocrine-disorders',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/neuropsychiatric-conditions',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/sense-organ-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/cardiovascular-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/respiratory-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/digestive-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/genitourinary-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/musculoskeletal-diseases',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/congenital-anomalies',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/oral-conditions',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/unintentional-injuries',
         'https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/intentional-injuries']
cause=['Infectious and parasitic diseases',
           'Respiratory infections',
           'Maternal conditions',
           'Perinatal conditions',
           'Nutritional deficiencies',
           'Malignant neoplasms',
           'Diabetes mellitus and endocrine disorders',
           'Neuropsychiatric conditions',
           'Sense organ diseases',
           'Cardiovascular diseases',
           'Respiratory diseases',
           'Digestive diseases',
           'Genitourinary diseases',
           'Musculoskeletal diseases',
           'Congenital anomalies',
           'Oral conditions',
           'Unintentional injuries',
           'Intentional injuries']

cause_type=['Communicable, maternal, perinatal and nutritional conditions',
           'Noncommunicable diseases',
           'Injuries']

gender=['All']
age=['0','1-4','5-14','15-24','25-34','35-54','55-74','75+']
age=['0,']

In [4]:
gender=['All']
age=['0','1-4','5-14','15-24','25-34','35-54','55-74','75+']

list_url=['https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/hiv-aids', 
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/malaria', 
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/tuberculosis', 
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/dengue', 
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/covid-19',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/brain-and-nervous-system-cancers',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/leukaemia',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/road-traffic-accidents',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/fires',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/drownings',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/natural-disasters',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/self-inflicted-injuries',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/violence',
          'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/war'
         ]

cause=['HIV/AIDS','Malaria','Tuberculosis','Dengue','Covid-19','Brain and nervous system cancers','Leukaemia','Road traffic accidents','Fires','Drownings',
       'Natural disasters','Self-inflicted injuries','Violence','War']

cause_type={'HIV/AIDS':'Infectious and parasitic diseases', 
            'Malaria':'Infectious and parasitic diseases', #sốt rét
            'Tuberculosis':'Infectious and parasitic diseases', #bệnh lao
            'Dengue':'Infectious and parasitic diseases', #sốt xuất huyết
            'Covid-19':'Infectious and parasitic diseases',
            'Brain and nervous system cancers':'Noncommunicable diseases', #ung thư não, hệ thần kinh
            'Leukaemia':'Noncommunicable diseases', #ung thư máu
            'Road traffic accidents':'Unintentional injuries', 
            'Fires':'Unintentional injuries',
            'Drownings':'Unintentional injuries',
            'Natural disasters':'Unintentional injuries', #thảm họa thiên nhiên
            'Self-inflicted injuries':'Intentional injuries', #tự tử
            'Violence':'Intentional injuries', #bạo lực
            'War':'Intentional injuries' #chiến tranh
           } 

In [6]:
len(list_url)
#for i in range(0,len(list_url)):
#    print(list_url[i],cause[i],cause_type[cause[i]])

14

In [9]:
#Đang làm
#len(list_url)
df=pd.DataFrame()
country=[]
for i in range(5,6):
    row={} #biến lưu các giá trị của 1 dòng
    driver.get(list_url[i])
    row['Cause type']=cause_type[cause[i]];
    row['Cause death']=cause[i];
    
    #chọn dạng biểu đồ
    driver.find_element(By.XPATH, '//*[@id="visualization_component_4_key"]').click()
    
    #chọn quốc gia
    #chỉ chọn cho lần chạy đầu tiên:
    if i==0:
        driver.implicitly_wait(30)
        driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key"]/input').click()
        driver.implicitly_wait(30)
        driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key_ul_level1"]/li[1]/label/input').click()
    
    for g in gender:
          #  row['Gender']=g
        #chọn giới tính
        driver.implicitly_wait(3)
        select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[5]')
        select = Select(select.find_element(By.CLASS_NAME,'form-control'))
        
        select.select_by_visible_text(g)
        
        #chọn nhóm tuổi
        for a in age:
            row['Age']=a
            driver.implicitly_wait(10)
            select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[4]')
            select = Select(select.find_element(By.CLASS_NAME,'form-control'))
            select.select_by_visible_text(a)
            
            #Filter
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="filters_section_indicatorlinechart_countries_key_filter_button"]').click()
            #đợi load
            time.sleep(10)
                
            #lăp lấy các bảng 
            for j in range(0,3):
                table_id = driver.find_element(By.ID, 'custom_table')
                cols = table_id.find_elements(By.TAG_NAME, "th")
                
                country=[]
                for c in cols:
                    country.append(c.text)
                country=country[1:]
                        
                #lấy số dòng
                rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]
                    
                for r in rows:
                    values=r.find_elements(By.TAG_NAME, "td")
                    row['Year']=values[0].text
                        
                    for k in range (1,len(values)):
                        row['Country']=country[k-1]
                        if values[k].text=='':
                            row['Number of deaths']=np.nan
                        else:
                            row['Number of deaths']=int(values[k].text.replace(' ',''))
                        print(row)
                        if df.empty:
                            df=pd.DataFrame.from_dict(row,orient='index').T
                        else:
                            df=pd.concat([df, pd.DataFrame.from_dict(row,orient='index').T])
                    
                
                if j!=2:
                    driver.find_element(By.XPATH, '//*[@id="next_button"]').click()
                    time.sleep(0.5)
                    table_id = driver.find_element(By.ID, 'custom_table')

    if i==0:
        df.to_csv(r'data.csv', index = None, header=True)
        df=pd.DataFrame()
    else:
        df.to_csv(r'data.csv', index = None, header=False,mode='a')
        df=pd.DataFrame()
                
                

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Albania', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Antigua and Barbuda', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Argentina', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Armenia', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Australia', 'Number of deaths': 1}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Austria

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Israel', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Italy', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Jamaica', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Japan', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Jordan', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Kazakhstan', 'Number of dea

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Suriname', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Sweden', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Switzerland', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Syrian Arab Republic', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'Tajikistan', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2020', 'Country': 'T

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Finland', 'Number of deaths': 1}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'France', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'French Guiana', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Georgia', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Germany', 'Number of deaths': 2}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Greece', 'Number of de

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Saint Vincent and the Grenadines', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Serbia', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Seychelles', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Singapore', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 'Slovakia', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '0', 'Year': '2019', 'Country': 

...

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2002', 'Country': 'Serbia', 'Number of deaths': 53}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2002', 'Country': 'Seychelles', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2002', 'Country': 'Singapore', 'Number of deaths': 4}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2002', 'Country': 'Slovakia', 'Number of deaths': 51}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2002', 'Country': 'Slovenia', 'Number of deaths': 23}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2002', 'Country': 'South Afri

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Dominica', 'Number of deaths': 0}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Dominican Republic', 'Number of deaths': 9}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Ecuador', 'Number of deaths': 20}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Egypt', 'Number of deaths': 128}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'El Salvador', 'Number of deaths': 14}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'E

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Poland', 'Number of deaths': 353}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Portugal', 'Number of deaths': 137}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Puerto Rico', 'Number of deaths': 27}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Republic of Korea', 'Number of deaths': 133}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001', 'Country': 'Republic of Moldova', 'Number of deaths': 3}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2001'

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Canada', 'Number of deaths': 331}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Chile', 'Number of deaths': 28}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'China, Hong Kong SAR', 'Number of deaths': 13}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Colombia', 'Number of deaths': 115}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Costa Rica', 'Number of deaths': 13}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country':

{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Netherlands', 'Number of deaths': 129}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'New Zealand', 'Number of deaths': 50}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Nicaragua', 'Number of deaths': nan}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'North Macedonia', 'Number of deaths': 13}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Country': 'Norway', 'Number of deaths': 52}
{'Cause type': 'Noncommunicable diseases', 'Cause death': 'Brain and nervous system cancers', 'Age': '75+', 'Year': '2000', 'Countr

# Khúc sau bỏ

In [ ]:
df=pd.DataFrame()
country=[]
for i in range(0,len(list_url)):
    row={} #biến lưu các giá trị của 1 dòng
    driver.get(list_url[i])
    row['Cause type']=cause_type[cause[i]];
    row['Cause death']=cause[i];
    
    #chọn dạng biểu đồ
    driver.find_element(By.XPATH, '//*[@id="visualization_component_4_key"]').click()
    
    #chọn quốc gia
    #chỉ chọn cho lần chạy đầu tiên:
    if i==0:
        driver.implicitly_wait(30)
        driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key"]/input').click()
        driver.implicitly_wait(30)
        driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key_ul_level1"]/li[1]/label/input').click()
    
    for g in gender:
          #  row['Gender']=g
        #chọn giới tính
        driver.implicitly_wait(3)
        select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[5]')
        select = Select(select.find_element(By.CLASS_NAME,'form-control'))
        
        select.select_by_visible_text(g)
        
        #chọn nhóm tuổi
        for a in ['All']:
            row['Age']=a
            driver.implicitly_wait(10)
            select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[4]')
            select = Select(select.find_element(By.CLASS_NAME,'form-control'))
            select.select_by_visible_text(a)
            
            #Filter
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="filters_section_indicatorlinechart_countries_key_filter_button"]').click()
            #đợi load
            time.sleep(10)
                
            #lăp lấy các bảng 
            table_id = driver.find_element(By.ID, 'custom_table')
            cols = table_id.find_elements(By.TAG_NAME, "th")
                
            country=[]
            for c in cols:
                country.append(c.text)
            country=country[1:]
            print(len(country))

In [ ]:
driver.get(url)

In [ ]:
#chọn dạng biểu đồ
driver.find_element(By.XPATH, '//*[@id="visualization_component_4_key"]').click()

In [ ]:
#chọn giới tính
driver.implicitly_wait(3)
select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[5]')
select = Select(select.find_element(By.CLASS_NAME,'form-control'))

In [ ]:
#Chọn gt làm nam
gender='Male'
select.select_by_visible_text(gender)

In [ ]:
#chọn nhóm tuổi
age='0'
driver.implicitly_wait(10)
select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[4]')
select = Select(select.find_element(By.CLASS_NAME,'form-control'))
select.select_by_visible_text(age)

In [ ]:
#chọn quốc gia
driver.implicitly_wait(30)
driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key"]/input').click()
driver.implicitly_wait(30)
driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key_ul_level1"]/li[1]/label/input').click()

In [ ]:
#Filter
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="filters_section_indicatorlinechart_countries_key_filter_button"]').click()
#đợi load
time.sleep(2)

In [ ]:
table_id = driver.find_element(By.ID, 'custom_table')
cols = table_id.find_elements(By.TAG_NAME, "th")


In [ ]:
#lấy số quốc gia
country=[]
for i in cols:
    country.append(i.text)
country=country[1:]
country

In [ ]:
#số dòng - bỏ header, header đã lưu vào biến col
rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]
len(rows)

In [ ]:
#lấy năm
#year=[]
#for i in range (1,11):
#    year.append(driver.find_element(By.XPATH,'//*[@id="custom_table_body"]/tr['+str(i)+']/td[1]').text)
#year

In [ ]:
driver.find_element(By.XPATH, '//*[@id="custom_table_body"]/tr[1]/td[2]').text

In [ ]:
df=pd.DataFrame()
for r in rows:
    v={}
    values=r.find_elements(By.TAG_NAME, "td")
    v['Year']=values[0].text
    v['Sex']='Male'
    v['Cause Type']='Communicable, maternal, perinatal and nutritional conditions'
    v['Cause Death']='Infectious and parasitic diseases'
    for i in range (1,len(values)):
        v['Country']=country[i-1]
        if values[i].text=='':
            v['Number of deaths']=np.nan
        else:
            v['Number of deaths']=int(values[i].text.replace(' ',''))
        print(v)
        if df.empty:
            df=pd.DataFrame.from_dict(v,orient='index').T
        else:
            df=pd.concat([df, pd.DataFrame.from_dict(v,orient='index').T])
        
    
    

In [ ]:
df=df.reset_index()
display(df)

In [ ]:
driver.find_element(By.XPATH, '//*[@id="next_button"]').click()
time.sleep(0.5)
table_id = driver.find_element(By.ID, 'custom_table')

In [ ]:
#lặp lại các bước
#số dòng - bỏ header, header đã lưu vào biến col
rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]

for r in rows:
    v={}
    values=r.find_elements(By.TAG_NAME, "td")
    v['Year']=values[0].text
    v['Sex']='Male'
    v['Cause Type']='Communicable, maternal, perinatal and nutritional conditions'
    v['Cause Death']='Infectious and parasitic diseases'
    for i in range (1,len(values)):
        v['Country']=country[i-1]
        
        # cần tạo 1 cái gì đó để tra xem quốc gia đó thuộc khu vực nào và tạo trường area
        if values[i].text=='':
            v['Number of deaths']=np.nan
        else:
            v['Number of deaths']=int(values[i].text.replace(' ',''))
        print(v)
        if df.empty:
            df=pd.DataFrame.from_dict(v,orient='index').T
        else:
            df=pd.concat([df, pd.DataFrame.from_dict(v,orient='index').T])

In [ ]:
driver.find_element(By.XPATH, '//*[@id="next_button"]').click()
time.sleep(0.5)
table_id = driver.find_element(By.ID, 'custom_table')

In [ ]:
#lặp lại các bước
#số dòng - bỏ header, header đã lưu vào biến col
rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]

for r in rows:
    v={}
    values=r.find_elements(By.TAG_NAME, "td")
    v['Year']=values[0].text
    v['Sex']='Male'
    v['Cause Type']='Communicable, maternal, perinatal and nutritional conditions'
    v['Cause Death']='Infectious and parasitic diseases'
    for i in range (1,len(values)):
        v['Country']=country[i-1]
        if values[i].text=='':
            v['Number of deaths']=np.nan
        else:
            v['Number of deaths']=int(values[i].text.replace(' ',''))
        print(v)
        if df.empty:
            df=pd.DataFrame.from_dict(v,orient='index').T
        else:
            df=pd.concat([df, pd.DataFrame.from_dict(v,orient='index').T])

In [ ]:
df 

# 2373 cho 1 nhóm tuổi của 1 nguyên nhân 1 giới tính :) 2373x8x32x3=1,822,464
Số vòng for cần chạy:

- For nguyên nhân

- For nhóm tuổi

- for giới tính

    CODE ở trên và truyền tham số lại